In [1]:
% run items.py
import pandas as pd 
import numpy as np

In [2]:
items_df = pd.DataFrame(items)
items_df

,border,color,form,letter
0,False,blue,square,E
1,True,red,circle,A
2,False,green,circle,B
3,True,red,diamond,E
4,True,red,diamond,E
5,False,blue,diamond,C
6,False,green,square,A
7,True,green,diamond,D
8,True,silver,star,C
9,False,silver,square,C


In [3]:
it = items_df.drop('border', axis=1)
tar = items_df['border']

In [55]:
it_ar = np.array(items_df.drop('border', axis=1))
tar_ar = np.array(items_df['border'])

In [ ]:
cols = np.array((it.columns))

In [ ]:
tree = tree_builder_numpy(it_ar, tar_ar)

In [ ]:
b

In [ ]:
split_candidates = np.array(it.columns)

In [ ]:
scs = split_candidates

In [ ]:
%%timeit
build_tree_alt(it, tar)

In [ ]:
%%timeit
tree_builder_numpy(it_ar, tar)

In [ ]:
b

In [ ]:
def tree_builder(data, target, split_candidates=None):
    scs = ['color', 'form', 'letter']
    if split_candidates is None:
        split_candidates = np.array([i for i in range(data.shape[1])])
        
    total_count = target.shape[0]
    true_count = target.sum()
    false_count = total_count - true_count 
    
    if true_count == 0:
        return False
    elif false_count == 0:
        return True

    if split_candidates.shape[0] == 0:
        return true_count > false_count

    best_split = find_best_split(data, target, split_candidates)
    
    split_candidates = np.array([split_candidate for split_candidate in split_candidates if split_candidate != best_split])
 
    nodes = np.unique(data[:, best_split])
    subtree_dict = dict()
    
    for node in nodes:
        sub_mask = data[:, best_split] == node
        data_sub = data[sub_mask]
        tar_sub = target[sub_mask]
        
        subtree_dict[node] = tree_builder(data_sub, tar_sub, split_candidates)
    
    subtree_dict[None] = true_count > false_count
    
    return(scs[best_split], subtree_dict)         

In [ ]:
def tree_builder(data, target,split_candidates):
    
    # needs changed 
    split_count = len(split_candidates)
    
    total_count = target.shape[0]
    true_count = target.sum()
    false_count = total_count - true_count

    
    if len(split_candidates) == 0:
        return true_count > false_count
        print('done')
    
    best_split_indx = find_best_split(data, target)
    
    best_split = split_candidates[best_split_indx]
    
    print('best split: {} \n'.format(best_split))
    
    if true_count == 0: return False
    if false_count == 0: return True
    
    # removes best_split from split _candidates
    split_candidates = [split_candidate
                    for split_candidate in split_candidates
                    if split_candidate != best_split]
    
    nodes = np.unique(data[:,best_split_indx])
    
    
    subtree_dict = dict()
    
    for node in nodes:
        sub_mask = data[:,best_split_indx] == node
        data_sub = data[sub_mask]
        tar_sub = target[sub_mask]
        
        subtree_dict[node] = tree_builder(data_sub, tar_sub, split_candidates)
    
    subtree_dict[None] = true_count > false_count
        
    return(best_split, subtree_dict)
    

In [35]:

def classify(tree, element):
    
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associted with that split
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    # get the subtree associated with the key
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)

In [ ]:
def class_proportions_alt(target):
    
    total_count = target.count()
    class_count = target.value_counts()
    return class_count / total_count

def target_proportions_alt(target):
        
    total_count = target.shape[0]
    class_count = np.bincount(target)
    
    return class_count / total_count


def entropy_alt(proportions):
    proportions = np.array(proportions)
    log_2_proportions = np.log2(proportions)
    return -proportions.dot(log_2_proportions)


def entropy_on_split_alt(dataframe, target, feature, debug=False):
    
    # split on feature
    unique_classes = dataframe[feature].unique()
    
    target_subsets = [
        target[dataframe[feature] == unique_class]
        for unique_class in unique_classes]
    
    if debug:
        print('Target Subsets:\n{}\n'.format(target_subsets))
        
    # calculate subset weights
    total = target.count()
    weights = [
        target_subset.count()/total
        for target_subset in target_subsets
    ]
    if debug:
        print('Weights:\n{}\n'.format(weights))
        
    # calculate target proprotions
    proportions = [
        class_proportions_alt(target_subset)
        for target_subset in target_subsets
    ]
    if debug: print('Proportions:\n{}\n'.format(proportions))
        
    # calculate the entropies
    entropies = [
        entropy_alt(proportion)
        for proportion in proportions
    ]
    
    if debug: print('Entropies:\n{}\n'.format(entropies))
        
    weighted_entropies = [
        weight*entropy
        for weight, entropy in zip(weights,entropies)
    ]
    return sum(weighted_entropies)

def classify_alt(tree, element):
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary 
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associated with that split
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    #get the subtree associated with the key 
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)


def find_best_split_alt(features, target, split_candidates):
    
    entropies = [
        entropy_on_split_alt(features, target, split_candidate)
        for split_candidate in split_candidates
    ]

    best_index = np.argmin(entropies)
    return split_candidates[best_index]


def build_tree_alt(features, target, split_candidates=None):
    #handle the case for split_candidates is None
    
    if split_candidates is None:
        split_candidates = list(features.columns)
        
    total_count = target.count()
    true_count = target.sum()
    false_count = total_count - true_count
    
    if false_count == 0: return True
    if true_count == 0: return False
    
    if split_candidates == []:
        return true_count > false_count
    
    # find the best attribute to split upon
    best_attribute = find_best_split_alt(features, target, split_candidates)

    # remove best attribute from list of split candidates
    split_candidates = [split_candidate
                       for split_candidate in split_candidates
                       if split_candidate is not best_attribute]
    
    best_attribute_uniques = features[best_attribute].unique()
    
    subtree_dictionary = dict()
    
    for best_attribute_unique in best_attribute_uniques:
        feat_subset_mask = features[best_attribute] == best_attribute_unique
        feat_subset = features[feat_subset_mask]
        target_subset = target[feat_subset_mask]
        
        subtree_dictionary[best_attribute_unique] = \
            build_tree_alt(feat_subset, target_subset, split_candidates)
            
    subtree_dictionary[None] = true_count > false_count
        
    return(best_attribute, subtree_dictionary)
    
            

In [34]:
it

,color,form,letter
0,blue,square,E
1,red,circle,A
2,green,circle,B
3,red,diamond,E
4,red,diamond,E
5,blue,diamond,C
6,green,square,A
7,green,diamond,D
8,silver,star,C
9,silver,square,C


In [47]:
def classify(tree, element):
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary 
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associated with that split
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    #get the subtree associated with the key 
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)

In [ ]:
build_tree_alt(it, tar)

# NEED

In [ ]:
def entropy(prob):
    # replacing 0 values with 1 to avoid issues with np.log2 returning -inf
    # will result in the same entropy
    zero_mask = prob == 0
    prob[zero_mask] = 1
    
    log_2_prob = np.log2(prob)
    return -prob * log_2_prob

In [ ]:
def entropy_split(data, target, feature, debug=False):
    
#   splitting data along a feature
    unique_class, class_counts = np.unique(data[:,feature], return_counts=True)
    
    weights = class_counts/ target.shape[0]
    
    if debug: print('Weights\n{}\n'.format(weights))
    # getting the split proportions
    splits = [target[data[:,feature] == uc] for uc in unique_class]
    
    prob = np.empty((unique_class.shape[0],2))
        
    for i, split in enumerate(splits):

        prob[i][0] = split.mean()
        prob[i][1] = 1 - split.mean()

    if debug: print('Probability\n{}\n'.format(prob))
        
    entropies = entropy(prob)
    
    if debug: print('entropies\n{}\n'.format(entropies))
    
    # transposing entropies shape so it can be multiplied with weights
    w_ent = entropies.T * weights
            
    return w_ent.sum()

In [ ]:
def find_best_split(data, target, split_candidates):
    split_entropies = [entropy_split(data, target, split_candidate) for split_candidate in split_candidates]
    
    splt_ent = np.array(split_entropies)
    sort_ent_indx = splt_ent.argsort()
    best_ent_indx = sort_ent_indx[0]

    return split_candidates[best_ent_indx]

In [ ]:
def find_best_split_two(data, target):
    
    split_entropies = []
    
    

In [ ]:
entropies = entropy_split(it_ar, tar_ar,1,debug=True)
entropies

In [ ]:
# other

In [ ]:
def find_best_split(data, target):
    
    split_entropies = []
    
    for i in range(data.shape[1]):
        ent = entropy_split(data, target, i)
        split_entropies.append(ent)
    
    splt_ent = np.array(split_entropies)
    return splt_ent.argsort()

def find_best_split(features, target, split_candidates):
    entropies = [
        entropy_on_split(features, target, split_candidate)
        for split_candidate in split_candidates
    ]
    best_index = np.argmin(entropies)
    return split_candidates[best_index]

In [82]:
class DecisionTree():
    
    def entropy(self, prob):
        '''
        
        '''
        # replacing - values with 1 to avoid issues with np.log2 returning - inf
        # both 0 and 1 represent pure splits so this change will not effect the result

        zero_mask = prob == 0
        prob[zero_mask] = 1

        log_2_prob = np.log2(prob)
        return -prob * log_2_prob

    def entropy_split(self, data, target, feature, debug=False):
    
#   splitting data along a feature
        unique_class, class_counts = np.unique(data[:,feature], return_counts=True)
    
        weights = class_counts/ target.shape[0]
    
        if debug: print('Weights\n{}\n'.format(weights))
    # getting the split proportions
        splits = [target[data[:,feature] == uc] for uc in unique_class]
    
        prob = np.empty((unique_class.shape[0],2))
        
        for i, split in enumerate(splits):

            prob[i][0] = split.mean()
            prob[i][1] = 1 - split.mean()

        if debug: print('Probability\n{}\n'.format(prob))
        
        entropies = self.entropy(prob)
    
        if debug: print('entropies\n{}\n'.format(entropies))
    
        # transposing entropies shape so it can be multiplied with weights
        w_ent = entropies.T * weights
            
        return w_ent.sum()
    
    def find_best_split(self, data, target, split_candidates):
        split_entropies = [self.entropy_split(data, target, split_candidate) for split_candidate in split_candidates]
    
        splt_ent = np.array(split_entropies)
        sort_ent_indx = splt_ent.argsort()
        best_ent_indx = sort_ent_indx[0]

        return split_candidates[best_ent_indx]

    def tree_builder(self, data, target, split_candidates=None):
        
        if split_candidates is None:
            split_candidates = np.array([i for i in range(data.shape[1])])
        
        total_count = target.shape[0]
        true_count = target.sum()
        
        false_count = total_count - true_count 
    
        if true_count == 0:
            return False
        elif false_count == 0:
            return True

        if split_candidates.shape[0] == 0:
            return true_count > false_count

        best_split = self.find_best_split(data, target, split_candidates)
    
        split_candidates = np.array([split_candidate for split_candidate in split_candidates if split_candidate != best_split])
 
        nodes = np.unique(data[:, best_split])
        subtree_dict = dict()
    
        for node in nodes:
            sub_mask = data[:, best_split] == node
            data_sub = data[sub_mask]
            tar_sub = target[sub_mask]
        
            subtree_dict[node] = self.tree_builder(data_sub, tar_sub, split_candidates)
    
        subtree_dict[None] = true_count > false_count
    
        return(self.features[best_split], subtree_dict)

    def fit(self, data, target):
        
        self.features = np.array(data.columns)
        
        data = np.array(data)
        target = np.array(target)
        
        tree = self.tree_builder(data, target)
        return tree
    
    def predict(self, data, target):
        pass

In [83]:
treebly = DecisionTree()

In [84]:
tree = treebly.fit(it, tar)

In [85]:
tree

('form',
 {None: True,
  'circle': ('color',
   {None: True,
    'blue': True,
    'green': ('letter', {'B': False, 'D': True, None: False}),
    'red': True}),
  'diamond': ('color',
   {None: True, 'blue': False, 'green': True, 'red': True}),
  'square': False,
  'star': True})

In [101]:
l = []
for row in it:
    l.append(row)

In [104]:
l

['color', 'form', 'letter']

In [110]:
obs = it.loc[0,:]

In [115]:
obs['form']

'square'

In [87]:
node, subtree = tree

In [ ]:
for ro

In [125]:
prediction = []
for i in range(it.shape[0]):
    prediction.append(predict(tree,it.loc[i,:]))

In [127]:
correct = []
for pred, act in zip (prediction, tar):
    correct.append(pred == act)
    

In [129]:
correct

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [116]:
def predict(tree,element):
    if tree in [True, False]:
        return tree
    
    node, branches = tree
    
    branch = element[node]
    
    if branch not in branches:
        branch = None
    
    subtree = branches[branch]
    
    return classify(subtree, element)

In [ ]:
def classify(tree, element):
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary 
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associated with that split
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    #get the subtree associated with the key 
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)